<a href="https://colab.research.google.com/github/brownt47/Python_Projects/blob/main/Faculty_Overloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from pytz import timezone

# DATA CLEANING

#### Import the Course Schedule data

In [ ]:
filename = 'CV_Class_Schedule_By_Department_S200106-2020-07-16_20210830_095625.csv'

In [ ]:
#loc = "C:\\Users\\User\\OneDrive - Georgia State University\\!!!Online Math Department\\Schedules for Online\\Fall 2021"
#loc = loc + "\\" + filename

In [ ]:
stats = pd.read_csv(filename, dtype={'seq_numb':str}, na_values="")

#### Campus Code dictionary

In [ ]:
campuses = ['PA','PF','PS','PC','PE', 'PN']
campuses_dict = {'PA':"Alpharetta",'PF':'Online','PS':'Decatur','PC':'Clarkston','PE':'Newton', 'PN':'Dunwoody'}

## Clean and Create Custom Columns

#### Convert column names to lowercase

In [ ]:
stats.columns = [x.lower() for x in stats.columns]
stats.columns = ['term', 'coll', 'crn', 'dept', 'sapr', 'xlst', 'ptrm', 'subj_crsnumb',
       'crse_title', 'seq_numb', 'schd', 'insm_code', 'crse_delvry', 'campus',
       'sess', 'credit_hr_low', 'credit_hr_high', 'credit_hrs', 'sest', 'ses#',
       'days', 'start_time', 'end_time', 'schd.1', 'loc', 'instructor',
       'spriden_id', 'fcst_code', 'enroll', 'max_enrl', 'xlst_enrl',
       'xlst_max_enrl', 'crse_status', 'comments', 'fees', 'detail code',
       'sec', 'ssbsect_voice_avail']

#### Assign Subject codes for courses (Math, Eng, CS)

In [ ]:
subject_list=stats.subj_crsnumb
insert_subject_list = []
for x in subject_list:
  insert_subject_list.append(str(x).split(" ")[0])
stats = stats.assign(subject=insert_subject_list)

#### Capture a list of all subjects

In [ ]:
subject_list = list(stats.subject.unique())

#### Capture Course-Section from subj_csrnumb

In [ ]:
stats['course_section'] = stats['subj_crsnumb'].str.split(" ",1,expand=True)[1]
stats['seq_numb'] = stats['seq_numb'].astype('str')
stats['seq_numb'] = stats['seq_numb'].str.zfill(3)
stats['course_section'] = stats['course_section']+ "-" +stats['seq_numb']

#### Assign Lecturer, Full-Time and Part-Time status for special case faculty

In [ ]:
lecturer_list = ['01 LAMAR, TOSHA','01 Malhotra, Charoo','01 Pendarvis, Denise','01 Marshall, Johnathan']
for lecturer in lecturer_list:
  stats['fcst_code']=np.where((stats['instructor']==lecturer),
                              'LC',
                              stats['fcst_code'])

In [ ]:
AD_list = ['01 Yeomans, Kevin']
for convert_AD in AD_list:
  stats['fcst_code']=np.where((stats['instructor']==convert_AD),
                              'FT',
                              stats['fcst_code'])

#### Create 'Part of Term' Column

In [ ]:
stats['part_of_term'] = 'full'

#### Create WLU with listing of xlisted courses with dropped duplicates
> ###### WLU = Work Load Units - xlisted courses shouldn't count in totals and  need to be zero'd out.

In [ ]:
crn_dups = list(stats.duplicated(subset=['xlst'],keep = 'last'))
stats = stats.assign(WLU = stats['credit_hr_low']*(crn_dups))

# Finding Overloaded Faculty

#### Select Columns for reporting

In [ ]:
data_columns = ['crn','ptrm','subject','xlst','subj_crsnumb',
       'campus', 'credit_hr_low',
       'instructor', 'fcst_code']

In [ ]:
teaching_load =stats[data_columns]
new_data_columns = ['crn','total courses','subject','xlst','subj_crsnumb',
       'campus', 'credit_hr_low',
       'instructor', 'fcst_code']
teaching_load.columns = new_data_columns


## Combine cross-listed courses

#### Create separate dataframe for non-cross-listed courses

In [ ]:
not_xlisted_courses = teaching_load[teaching_load['xlst'].isnull()]
#not_xlisted_courses

#### Create separate dataframe collapsing cross-listed courses

In [ ]:
xlisted_courses = teaching_load[teaching_load['xlst'].notnull()].groupby('xlst').max()

#### Union the non-crosslisted courses with collapsed crosslisted courses

In [ ]:
courses = pd.concat([not_xlisted_courses , xlisted_courses])

## Groupby instructor and sum up all courses and credit hours


In [ ]:
overload_df=courses
Overload_sums= overload_df.groupby(['instructor']).agg(
   total_hours=pd.NamedAgg(column="credit_hr_low", aggfunc="sum"), 
   total_courses = pd.NamedAgg(column = 'total courses', aggfunc = 'sum'),
   instructor = pd.NamedAgg(column = 'instructor', aggfunc = 'first'),
   fcst_code = pd.NamedAgg(column = 'fcst_code', aggfunc = 'first')
   )


### Create Overload hours column

###### Remove unstaffed courses

In [ ]:
staff_filter = Overload_sums['instructor']!='01 Staff'
Overload_sums = Overload_sums[staff_filter]


##### Lecturers are overloaded if they teach at least 6 courses and more than 18 hours in any semester*
##### *Senior Lecturers are overloaded if they teach at least 5 courses and more than 15 hours in spring semesters

In [ ]:
lecturer_filter = Overload_sums['fcst_code']=='LC'
lecturer_df = Overload_sums[lecturer_filter]
lecturer_df.assign(fcst_code = 'LC')
lecturer_df=lecturer_df.assign(overload_hours = lecturer_df['total_hours']-18)
lecturer_df=lecturer_df.assign(overloaded = (lecturer_df['overload_hours']>0) & (lecturer_df['total_courses']>=6))

##### Tenure-Track faculty  are overloaded if they teach at least 5 courses and more than 15 hours in fall semesters
##### Tenure-Track faculty  are overloaded if they teach at least 4 courses and more than 12 hours in spring semesters

In [ ]:
tenure_track_filter = Overload_sums['fcst_code']!='LC'
tenure_track_df = Overload_sums[tenure_track_filter]
tenure_track_df=tenure_track_df.assign(overload_hours = tenure_track_df['total_hours']-15)
tenure_track_df=tenure_track_df.assign(overloaded = (tenure_track_df['overload_hours']>0) & (tenure_track_df['total_courses']>=5))


### Combine results for tenure-track faculty and lecturers

In [ ]:
Overload_sums = pd.concat([lecturer_df , tenure_track_df])

### Listing of Overloaded Faculty

In [ ]:

Overload_filter = (Overload_sums['overloaded']==True)
Overload_sums = Overload_sums[Overload_filter].sort_values(['overload_hours','total_courses','total_hours','fcst_code'],ascending=False)

In [ ]:
Overload_sums

,total_hours,total_courses,instructor,fcst_code,overload_hours,overloaded
instructor,,,,,,
"01 Cohen, Blair",20,10,"01 Cohen, Blair",FT,5,True
"01 Aslam, Mohammad",19,7,"01 Aslam, Mohammad",FT,4,True
"01 Pruvenok, Robert",19,7,"01 Pruvenok, Robert",FT,4,True
"01 Pendarvis, Denise",21,8,"01 Pendarvis, Denise",LC,3,True
"01 China, Ervin",18,8,"01 China, Ervin",FT,3,True
"01 Powell, Virginia",18,7,"01 Powell, Virginia",FT,3,True
"01 Bennekin, Kimberly",18,6,"01 Bennekin, Kimberly",FT,3,True
"01 Bennett, Kim",18,6,"01 Bennett, Kim",FT,3,True
"01 Du, Hong",18,6,"01 Du, Hong",FT,3,True


### List of names of overloaded faculty

In [ ]:
overloaded_faculty = list(Overload_sums['instructor'])

In [ ]:
overloaded_faculty

['01 Cohen, Blair',
 '01 Aslam, Mohammad',
 '01 Pruvenok, Robert',
 '01 Pendarvis, Denise',
 '01 China, Ervin',
 '01 Powell, Virginia',
 '01 Bennekin, Kimberly',
 '01 Bennett, Kim',
 '01 Du, Hong',
 '01 McBride, Christopher',
 '01 Hunter, Rodney',
 '01 Service, Angela',
 '01 Sheikh Ansari, Forough',
 '01 Chung, Michelle',
 '01 Bingi, Srilatha',
 '01 Rouhani, Behnaz',
 '01 Chowdhury, Ashraful',
 '01 Tapp, Marvelyn',
 '01 Weltlich, Sharon']

In [ ]:
len(overloaded_faculty)

# Create Excel export object

#### Create filename for report with timestamp

In [ ]:
eastern = timezone('US/Eastern')
dateTimeObj = datetime.now(eastern)
datestamp = dateTimeObj.strftime("%Y%b%d_%I_%M%p")
report_name = "Faculty_Overloads.xlsx" #+ datestamp + ".xlsx"
print(report_name)

Faculty_Overloads.xlsx


In [ ]:
writer = pd.ExcelWriter(report_name)

In [ ]:
# book object to format
workbook = writer.book  

In [ ]:
sheetname = 'Overloads'
Overload_sums.to_excel(writer,sheetname,index=False)

In [ ]:
overload_courses = stats

## Create Listing of CRNs and other data for Overload Form Report per instructor

In [ ]:
 #dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0) 

In [ ]:

temp_list = []

start_col=0
start_row=0
sheetname = 'CRN_listings'
for faculty in overloaded_faculty:
  filter = (overload_courses['instructor']==faculty)
  temp_list = overload_courses[filter][['instructor','crn','subject','course_section','credit_hr_low','WLU','start_time','days','part_of_term','enroll','xlst']]
  temp_list.to_excel(writer,sheetname,startrow=start_row,startcol=start_col,index=False)
  start_col = start_col+15
  if ((start_col%30)==0 and (start_col!=0)):
    start_row = start_row + 15
    start_col = 0

In [ ]:
writer.save()